# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import seaborn as sns

from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

## Chargement des données

In [ ]:
# Chargement depuis parquet
df = pd.read_parquet("../data/clean/lille_2022.parquet")

# Filtrer biens avec exactement 4 pièces principales
df_4p = df[df['Nombre pieces principales'] == 4].copy()
#df_4p = df

## préparation des données immobilières


#### `colonnes_a_garder`:
- Liste les colonnes utiles pour l’analyse des logements (surface, nombre de pièces, valeur, type).

#### `df_logements = df_4p[colonnes_a_garder].copy()`:
- Crée un nouveau DataFrame avec uniquement ces colonnes, afin de travailler sur un sous-ensemble propre.

#### Calcul de prix_m2 pondéré:
- Le prix au m² est ici calculé en prenant en compte la surface bâtie mais aussi une part pondérée de la surface du terrain (30%),
car la surface du terrain influence la valeur totale, surtout pour les maisons.

#### Gestion des valeurs manquantes:
- `.fillna(0)` remplace les éventuelles valeurs manquantes de Surface terrain par 0,
évitant ainsi une division par NaN ou une erreur lors du calcul.



In [ ]:
colonnes_a_garder = [
    'Surface reelle bati',
    'Nombre pieces principales',
    'Nombre de lots',
    'Valeur fonciere',
    'Surface terrain',
    'Code type local',
]

# logements
df_logements = df_4p[colonnes_a_garder].copy()

#Calcul du prix au m² pondéré (surface bâtie + 0.3 * surface terrain) 
df_logements['prix_m2'] = df_logements['Valeur fonciere'] / df_logements['Surface reelle bati'] + (0.3 * df_logements['Surface terrain'].fillna(0))

### Vérifier les types

In [ ]:
df_logements.dtypes

## Nettoyage et filtrage des données immobilières

Suppression des valeurs aberrantes communes
- Ce bloc filtre les logements pour garder uniquement ceux avec des valeurs plausibles :
- Valeur foncière entre 500 € et 1,5 million €
- Surface bâtie entre 20 et 200 m² (exclut les logements trop petits ou gigantesques)
- Prix au m² entre 500 € et 10 000 € (exclut les prix absurdes)

Filtres spécifiques aux maisons
- La variable is_maison identifie les maisons (Code type local == 1).
- On conserve tous les logements qui ne sont pas des maisons, ainsi que
- Les maisons seulement si leur surface terrain est raisonnable, entre 100 et 1500 m².
Cela permet d’exclure des maisons avec des terrains très petits ou trop grands qui pourraient fausser le modèle.

Traitement des valeurs manquantes dans 'Surface terrain' pour les appartements
- Pour les appartements (Code type local == 2), la surface terrain est souvent absente (NaN).
- Ici, on remplace ces NaN par 0, car les appartements n’ont généralement pas de terrain.
- Cela évite des erreurs lors des calculs ou entraînements de modèles.

Pourquoi ces étapes sont importantes ?
- Elles nettoient les données en éliminant les cas extrêmes ou incohérents.
- Elles améliorent la qualité d'entraînement du modèle en gardant un échantillon représentatif.
- Elles prennent en compte les spécificités des types de logements pour des règles adaptées.

In [ ]:
# Supprimer les valeurs aberrantes communes
df_logements = df_logements[
    (df_logements["Valeur fonciere"] > 500) &
    (df_logements["Valeur fonciere"] < 1_500_000) &
    (df_logements["Surface reelle bati"] >= 20) &
    (df_logements["Surface reelle bati"] <= 200) &
    (df_logements["prix_m2"] > 500) &
    (df_logements["prix_m2"] < 10_000)
]

# Appliquer les filtres spécifiques aux maisons uniquement
is_maison = df_logements["Code type local"] == 1
df_logements = df_logements[
    ~is_maison | (  # On garde tout sauf les maisons, OU les maisons avec terrain entre 100 et 1500 m²
        (df_logements["Surface terrain"] >= 100) & (df_logements["Surface terrain"] <= 1500)
    )
]

# Remplacer Surface terrain NaN par 0 pour les appartements (Code type local == 1)
df_logements.loc[
    (df_logements["Code type local"] == 2) & (df_logements["Surface terrain"].isna()),
    "Surface terrain"
] = 0

## 📊 Visualisation des surfaces bâties
Ce graphique compare la surface réelle bâtie pour deux types de logements :

- 🏢 Appartements
- 🏠 Maisons

Deux histogrammes sont affichés côte à côte pour permettre une comparaison visuelle directe.

### 🧱 Répartition des surfaces
- À gauche : la distribution des surfaces bâties des appartements, en bleu ciel.
- À droite : celle des maisons, en rouge saumon.
- Chaque barre représente le nombre de logements dans une tranche de surface donnée.

### 📍 Médiane visible
- Une ligne pointillée est tracée dans chaque histogramme pour indiquer la médiane de la surface bâtie.
    - 🟦 Ligne bleue pour les appartements
    - 🟥 Ligne rouge pour les maisons

Cela permet de voir si la répartition est équilibrée ou non autour de cette valeur centrale.

### 📏 Objectif du graphique
Ce double histogramme aide à :
- Comprendre les différences de taille entre maisons et appartements.
- Visualiser si certains types de logements sont plus petits ou plus grands en moyenne.
- Détecter d’éventuelles valeurs extrêmes ou des concentrations dans certaines plages.

### 🧠 Intérêt de cette comparaison
C’est une étape importante dans l’analyse car elle :
- Justifie les traitements séparés des maisons et appartements (modèles différents, filtres spécifiques).
- Donne des repères pour identifier les valeurs aberrantes ou peu fréquentes.
- Aide à mieux calibrer les modèles de prédiction du prix au m² en tenant compte du type de logement.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

# Paramètres communs
bins = 30
edge_color = 'white'

# appartements surface
axes[0].hist(df_logements[df_logements['Code type local'] == 2]['Surface reelle bati'], bins=bins, color='skyblue', edgecolor=edge_color)
axes[0].set_title("Répartition des Surface reelle bati - Appartements")
axes[0].set_xlabel("Surface reelle bati")
axes[0].set_ylabel("Nombre de logements")
axes[0].grid(True)

# Maisons surface
axes[1].hist(df_logements[df_logements['Code type local'] == 1]['Surface reelle bati'], bins=bins, color='salmon', edgecolor=edge_color)
axes[1].set_title("Répartition des Surface terrain - Maisons")
axes[1].set_xlabel("Surface reelle bati")
axes[1].grid(True)

# Ligne médiane
axes[0].axvline(df_logements[df_logements['Code type local'] == 2]['Surface reelle bati'].median(), color='blue', linestyle='--', label='Médiane')
axes[1].axvline(df_logements[df_logements['Code type local'] == 1]['Surface reelle bati'].median(), color='red', linestyle='--', label='Médiane')
axes[0].legend()
axes[1].legend()

## 💶 Répartition des prix au m² par type de logement
Ce graphique compare les prix au m² pour deux types de logements :

- 🏢 Appartements
- 🏠 Maisons

Deux histogrammes sont présentés côte à côte pour observer les différences de distribution.

## 📊 Que montre le graphique ?
- À gauche : la distribution des prix au m² pour les appartements.
- À droite : celle des maisons.
- Chaque barre représente le nombre de logements dans une tranche de prix au m².

## 📍 Ligne de médiane
- Une ligne pointillée indique la médiane du prix au m² :
    - 🔵 Ligne bleue pour les appartements
    - 🔴 Ligne rouge pour les maisons

Cela permet de voir rapidement si les prix sont centrés ou asymétriques.

## 🧠 Pourquoi c’est utile ?
- Permet de comparer visuellement les niveaux de prix entre maisons et appartements.
- Aide à détecter des valeurs extrêmes (logements très chers ou très bon marché).
- Justifie encore une fois l’intérêt de modéliser séparément les maisons et les appartements.
- Peut servir de base pour définir des seuils de nettoyage ou d’analyse (valeurs aberrantes, segments premium, etc.)

In [ ]:
#appartements
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

# Paramètres communs
bins = 30
edge_color = 'white'

# appartements prix_m2
axes[0].hist(df_logements[df_logements['Code type local'] == 2]['prix_m2'], bins=bins, color='skyblue', edgecolor=edge_color)
axes[0].set_title("Répartition des prix_m2 - Appartements")
axes[0].set_xlabel("prix_m2")
axes[0].set_ylabel("Nombre de logements")
axes[0].grid(True)

# Maisons prix_m2
axes[1].hist(df_logements[df_logements['Code type local'] == 1]['prix_m2'], bins=bins, color='salmon', edgecolor=edge_color)
axes[1].set_title("Répartition des prix_m2 - Maisons")
axes[1].set_xlabel("prix_m2")
axes[1].grid(True)

# Ligne médiane
axes[0].axvline(df_logements[df_logements['Code type local'] == 2]['prix_m2'].median(), color='blue', linestyle='--', label='Médiane')
axes[1].axvline(df_logements[df_logements['Code type local'] == 1]['prix_m2'].median(), color='red', linestyle='--', label='Médiane')
axes[0].legend()
axes[1].legend()

## 📈 Analyse de corrélation – Appartements
Cette étape permet d’analyser les relations statistiques entre le prix au m² et les autres variables pour les appartements uniquement.

### 🔍 Objectif
Identifier quelles caractéristiques influencent le plus le prix au m².
Cela nous aide à :

- Comprendre les facteurs importants dans la valorisation immobilière 🏢
- Sélectionner les variables les plus pertinentes pour l'entraînement du modèle 🔬

### 📊 Résultat attendu
Une liste triée de corrélations avec la variable cible prix_m2.
Chaque ligne montre :

- 🔢 Le nom de la variable
- 🔁 Une valeur de corrélation comprise entre -1 et +1

|Corrélation|	Interprétation
|---|---
|🔵 Proche de +1|	Forte corrélation positive (la variable augmente avec le prix)
|🔴 Proche de -1|	Forte corrélation négative (la variable diminue quand le prix augmente)
|⚪ Proche de 0|	Aucune corrélation (peu ou pas de lien)

### 🧠 Pourquoi c’est utile ?
- ✅ Pour la sélection de variables lors du machine learning
- ⚠️ Pour repérer les variables inutiles ou redondantes
- 🔎 Pour comprendre les leviers économiques dans le marché immobilier des appartements

In [ ]:
print("Corrélations Appartements :")
print(df_logements.corr()["prix_m2"].sort_values(ascending=False))

## 🔥 Visualisation de la Corrélation : Matrice Heatmap

Ce bloc de code permet de visualiser la force des liens entre plusieurs variables du dataset immobilier, sous forme de carte de chaleur (heatmap).

### 🎯 Objectif
Afficher de manière visuelle les corrélations entre :
- prix_m2 (cible)
- Valeur fonciere
- Code type local (1 = maison, 2 = appartement)
- Surface terrain
- Surface reelle bati
- Nombre pieces principales

### 🧪 Pourquoi c’est utile ?
- Repérer rapidement quelles variables sont pertinentes pour expliquer le prix
- Détecter des corrélations suspectes ou redondantes (ex : surface vs valeur foncière)
- Affiner la sélection des variables pour l'entraînement de modèles machine learning 🧠

In [ ]:
corr = df_logements[["prix_m2", "Valeur fonciere", "Code type local",
           "Surface terrain", "Surface reelle bati", "Nombre pieces principales"]].corr()

sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Matrice de corrélation")
plt.show()

## Préparer les données pour l'entraînement

In [ ]:
# Variables explicatives
features = ["Valeur fonciere", "Code type local", "Surface terrain", "Surface reelle bati"]
X = df_logements[features]

# Variable cible
y = df_logements["prix_m2"]

# Division en jeu d'entraînement (80%) et test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Entraîner les modèles de base avec scikit-learn

In [ ]:
# Modèles de base
lr = LinearRegression()
dt = DecisionTreeRegressor(random_state=42)
rf = RandomForestRegressor(random_state=42)

# Entraînement
lr.fit(X_train, y_train)
dt.fit(X_train, y_train)
rf.fit(X_train, y_train)

## Optimiser les modèles d’arbres avec GridSearchCV

In [ ]:
# Optimisation DecisionTreeRegressor
param_dt = {
    "max_depth": [3, 5, 10, None],
    "min_samples_split": [2, 5, 10]
}

grid_dt = GridSearchCV(DecisionTreeRegressor(random_state=42), param_dt, cv=5, scoring='neg_mean_squared_error')
grid_dt.fit(X_train, y_train)
dt_best = grid_dt.best_estimator_

# 🔍 Optimisation RandomForestRegressor
param_rf = {
    "n_estimators": [50, 100],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5]
}

grid_rf = GridSearchCV(RandomForestRegressor(random_state=42), param_rf, cv=5, scoring='neg_mean_squared_error')
grid_rf.fit(X_train, y_train)
rf_best = grid_rf.best_estimator_

## Ajouter un modèle moderne : XGBRegressor

In [ ]:
# Modèle XGBoost
xgb = XGBRegressor(random_state=42, eval_metric='rmse')
xgb.fit(X_train, y_train)

## Test sur divers models (valeurs global)

### 🧾 Rappel : qu’est-ce que le MSE ?

Le MSE (Mean Squared Error) mesure l’écart moyen au carré entre les vraies valeurs (y_test) et les valeurs prédites (y_pred).
- Un MSE proche de 0 = bonnes prédictions.
- Un MSE élevé = le modèle se trompe souvent ou fortement.
- ⚠️ Comme l’erreur est au carré, les grosses erreurs pèsent très lourd.

In [ ]:
# Regrouper les modèles
models = {
    "Linear Regression": lr,
    "Decision Tree (base)": dt,
    "Decision Tree (grid)": dt_best,
    "Random Forest (base)": rf,
    "Random Forest (grid)": rf_best,
    "XGBoost": xgb
}

# Évaluer chaque modèle sur le test
results = []
for name, model in models.items():
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results.append({"Modèle": name, "MSE": mse, "RMSE": rmse, "MAE": mae, "R2 Score": r2})

# Tableau comparatif global
results_df = pd.DataFrame(results).sort_values("MSE")
print("🔎 Résultats globaux (tous types confondus) :")
print(results_df)


Résultats par appartement et maison si Code type local est présent :

## 📊 Résultats Globaux (Tous types de logements)
|🔢 Rang|🧠 Modèle|🎯 MSE (erreur quadratique moyenne)|
|---|---|---|
|🥇 1|	Random Forest (base)|	23 246
|🥈 2|	XGBoost	|25 710
|🥉 3|	Random Forest (grid search)|	30 591
|4|	Decision Tree (base)|	46 314
|5|	Decision Tree (grid search)|	46 314
|6|	Linear Regression|	85 739

### 🧠 Interprétation rapide :
- ✅ Le meilleur modèle est le Random Forest (base) avec le MSE le plus bas (≈ 23 000), ce qui signifie qu’il prédit les prix au m² avec le moins d’erreurs moyennes.
- 🤖 XGBoost arrive en 2ᵉ position, assez proche en performance, ce qui confirme sa robustesse.
- ⚙️ Le grid search n’a pas amélioré Random Forest ni Decision Tree ici — probablement à cause d’un espace d’hyperparamètres sous-optimal ou un surapprentissage.
- 📉 Linear Regression est le moins performant, avec un MSE bien plus élevé — ce modèle linéaire simple ne capture pas bien la complexité du marché immobilier.

### 🧪 Pourquoi c’est important ?
- Le MSE (Mean Squared Error) mesure l’écart quadratique moyen entre les valeurs prédites et réelles. Plus il est bas, plus les prédictions sont précises.
- En comparant plusieurs modèles, on peut identifiez celui qui s’adapte le mieux à vos données.

## Test sur divers models (valeurs séparé)

### ❓ Pourquoi séparer maisons et appartements ?

### 1. Des caractéristiques différentes

|Variable|	Appartements|	Maisons
|---|---|---|
|Surface terrain|	Souvent nulle ou absente|	Très variable et importante
|Nombre pièces|	Plus faible en général|	Plus élevé
|Prix/m²|	Souvent plus élevé|	Moins élevé (car plus de m²)

### 2. Des comportements prix ≠
- Le prix au m² dépend différemment des variables selon le type :
- Pour un appartement, la surface du terrain est inutile.
- Pour une maison, elle peut être cruciale.
- Les interactions entre variables ne sont pas les mêmes.

### 3. Moins de bruit pour l'entraînement
- Si tu mélanges, le modèle apprend un compromis flou :
    - Il risque d’être moyennement bon partout, mais excellent nulle part.
- En séparant :
    - Le modèle se spécialise sur un type unique de structure de données.
    - Donc les performances sont nettement meilleures.

In [ ]:
results_maison = []
results_appartements = []

#séparation du dataset
df_maison = df_logements[df_logements["Code type local"] == 1]
df_appartement = df_logements[df_logements["Code type local"] == 2]

# Variables explicatives
features = ["Valeur fonciere", "Surface terrain", "Surface reelle bati"]

X_maison = df_maison[features]
X_appartement = df_appartement[features]

# Variable cible
y_maison = df_maison["prix_m2"]
y_appartement = df_appartement["prix_m2"]

# Division en jeu d'entraînement (80%) et test (20%)
X_train_maison, X_test_maison, y_train_maison, y_test_maison = train_test_split(X_maison, y_maison, test_size=0.2, random_state=42)
X_train_appartement, X_test_appartement, y_train_appartement, y_test_appartement = train_test_split(X_appartement, y_appartement, test_size=0.2, random_state=42)

# Modèles de base
lr = LinearRegression()
dt = DecisionTreeRegressor(random_state=42)
rf = RandomForestRegressor(random_state=42)

# Entraînement
lr.fit(X_train, y_train)
dt.fit(X_train, y_train)
rf.fit(X_train, y_train)


lr_maison = lr.fit(X_train_maison, y_train_maison)
lr_appartement = lr.fit(X_train_appartement, y_train_appartement)

dt_maison = dt.fit(X_train_maison, y_train_maison)
dt_appartement = dt.fit(X_train_appartement, y_train_appartement)

rf_maison = rf.fit(X_train_maison, y_train_maison)
rf_appartement = rf.fit(X_train_appartement, y_train_appartement)

# Optimisation DecisionTreeRegressor
param_dt = {
    "max_depth": [3, 5, 10, None],
    "min_samples_split": [2, 5, 10]
}

grid_dt = GridSearchCV(DecisionTreeRegressor(random_state=42), param_dt, cv=5, scoring='neg_mean_squared_error')

grid_dt_maison = grid_dt.fit(X_train_maison, y_train_maison)
grid_dt_appartement = grid_dt.fit(X_train_appartement, y_train_appartement)

dt_best_maison = grid_dt_maison.best_estimator_
dt_best_appartement = grid_dt_appartement.best_estimator_

# 🔍 Optimisation RandomForestRegressor
param_rf = {
    "n_estimators": [50, 100],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5]
}

grid_rf = GridSearchCV(RandomForestRegressor(random_state=42), param_rf, cv=5, scoring='neg_mean_squared_error')

grid_rf_maison = grid_rf.fit(X_train_maison, y_train_maison)
grid_rf_appartement = grid_rf.fit(X_train_appartement, y_train_appartement)

rf_best_maison = grid_rf_maison.best_estimator_
rf_best_appartement = grid_rf_appartement.best_estimator_

# Modèle XGBoost
xgb = XGBRegressor(random_state=42, eval_metric='rmse')

xgb_maison = xgb.fit(X_train_maison, y_train_maison)
xgb_appartement = xgb.fit(X_train_appartement, y_train_appartement)

models_maison = {
    "Linear Regression": lr_maison,
    "Decision Tree (base)": dt_maison,
    "Decision Tree (grid)": dt_best_maison,
    "Random Forest (base)": rf_maison,
    "Random Forest (grid)": rf_best_maison,
    "XGBoost": xgb_maison
}

models_appartement = {
    "Linear Regression": lr_appartement,
    "Decision Tree (base)": dt_appartement,
    "Decision Tree (grid)": dt_best_appartement,
    "Random Forest (base)": rf_appartement,
    "Random Forest (grid)": rf_best_appartement,
    "XGBoost": xgb_appartement
}

for name, model in models_maison.items():
    y_pred = model.predict(X_test_maison)
    mse = mean_squared_error(y_test_maison, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_maison, y_pred)
    r2 = r2_score(y_test_maison, y_pred)
    results_maison.append({"Modèle": name, "MSE": mse, "RMSE": rmse, "MAE": mae, "R2 Score": r2})
sub_df_maison = pd.DataFrame(results_maison).sort_values("MSE")
print(f"\n🏘️ Résultats pour maison :")
print(sub_df_maison)

for name, model in models_appartement.items():
    y_pred = model.predict(X_test_appartement)
    mse = mean_squared_error(y_test_appartement, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_appartement, y_pred)
    r2 = r2_score(y_test_appartement, y_pred)
    results_appartements.append({"Modèle": name, "MSE": mse, "RMSE": rmse, "MAE": mae, "R2 Score": r2})
sub_df_appartement = pd.DataFrame(results_appartements).sort_values("MSE")
print(f"\n🏘️ Résultats pour appartement :")
print(sub_df_appartement)

## 🏠 Résultats pour les Maisons
|🏆 Rang|	🧠 Modèle|	🎯 MSE (Erreur quadratique moyenne)
|---|---|---
|🥇 1|	XGBoost|	34 159
|🥈 2|	Decision Tree (base)|	61 008
|🥉 3|	Decision Tree (grid)|	61 008
|4|	Linear Regression|	88 679
|5|	Random Forest (base)|	98 844
|6|	Random Forest (grid)|	98 844

## ✅ Interprétation (Maisons) :
- XGBoost est clairement le plus performant (MSE le plus bas, R² = 0.95).
- Les arbres de décision sont corrects, mais moins précis.
- Les forêts aléatoires sont ici en retrait, avec un MSE plus élevé malgré leur complexité.
- La régression linéaire est la moins adaptée à ce type de bien, en raison de la complexité des relations entre terrain, bâti et valeur foncière.

## 🏢 Résultats pour les Appartements
|🏆 Rang|	🧠 Modèle|	🎯 MSE (Erreur quadratique moyenne)
|---|---|---
|🥇 1|	Decision Tree (base)|	46 346
|🥈 2|	Decision Tree (grid)|	46 346
|🥉 3|	Random Forest (base)|	50 360
|4|	Random Forest (grid)|	50 360
|5|	XGBoost|	71 256|
|6|	Linear Regression|	120 883

## ✅ Interprétation (Appartements) :
- L’arbre de décision simple surpasse tous les autres ici (MSE le plus bas, R² = 0.967).
- Random Forest reste performant, notamment en MAE.
- XGBoost, bien que souvent fort, n’est pas optimal sur ce sous-ensemble.
- La régression linéaire est clairement dépassée pour modéliser les prix des appartements.

## 🧠 Pourquoi séparer Appartements & Maisons ❓
|⚠️ Problème si on mélange|	✅ Avantage en séparant|
|---|---
|Terrain quasi nul pour appartements, très variable pour maisons|	Chaque modèle apprend les structures propres à chaque type de bien
|Mélange de distributions → biais d’entraînement|	Meilleure précision sur chaque sous-groupe
|Calcul de prix au m² faussé|	Meilleur MSE, MAE et R² par type de logement


## Sauvegarde

In [ ]:
import json

os.makedirs("../models/appartement", exist_ok=True)
os.makedirs("../models/maison", exist_ok=True)

print(results_appartements)

result = {
    "maison": results_maison,
    "appartement": results_appartements
}

# Sauvegarde des données dans un fichier JSON
with open("resultats_modeles_maison_appartement.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)
    
    
for name, model in models_maison.items():
    joblib.dump(model, f"../models/maisons/modele_maison_{name}.joblib")
print("✅ Modèles maisons sauvegardé sous 'models/maisons'")
    
for name, model in models_appartement.items():
    joblib.dump(model, f"../models/appartement/modele_appartement_{name}.joblib")
print("✅ Modèles appartements sauvegardé sous 'models/appartements'")